'''
    IAM handwritten data has given official split of data at line level.
    This data split is different from what we r using to train PHOSCNET.
    
    Below scripts finds train,test and validation data from yolov3.
    
    
    flow is as below.
    
    1.from official line level split it identify PHOSC original train,test,valid data at word level.
    
    2. Once it is done for above train,test,valid data corresponding yolov3 crop counter parts are identified.
    
    3. These yolov3 crop counter parts are then divided into official train-test-val
    
    4. Create a file offIamYoloV3CropSplit.pickle which identify each yolov3 crops to its corrresponding train.test val split.
    
    5. This moves yolov3 crops into their respective train,test,val split. This split is according to IAM handwritten official split.
    
    6. This also creates a .csv script helpfull for training PHOSCNet
    
'''

In [ ]:
"""
    THIS PART USES data_14_april.csv for line segmwntation
"""


In [1]:
import pandas as pd
import os
from PIL import Image, ImageDraw

import cv2

In [2]:
"""
    THIS IS OFFICIAL SPLIT FROM IAM DATA WHICH CONTAINS SPLIT LINES.
    
    IDEA IS TO USE THIS LINE NAMES TO IDENTIFY CROP BELONGS TO WHICH LINE.
    
    THE FILE data_14_.csv contains co-ordinate as well as the crop name from .xml file
    
    use this crop name to derive line no eg. g06-031n-04-05 here line no is 04 and word 
    
    no is 05. so removing last -05 and using remaining g06-031n-04 can let us know where it belong
    
    i.e. train,test,validation

"""


basePath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/data/"

csvpath=basePath+"/dataset/data_14_april.csv"
imgPath="./data/dataset/forms/"


basePath2="/home/aniketag/Documents/phd/PHOSC-Zero-Shot-Word-Recognition-main/data/"
offTrainSplit=basePath2+"/newFilterData/largeWriterIndependentTextLineRecognitionTask/trainset.txt"

offVal1Split=basePath2+"/newFilterData/largeWriterIndependentTextLineRecognitionTask/validationset1.txt"
offVal2Split=basePath2+"/newFilterData/largeWriterIndependentTextLineRecognitionTask/validationset2.txt"

offTestSplit=basePath2+"/newFilterData/largeWriterIndependentTextLineRecognitionTask/testset.txt"

with open(offTrainSplit) as f:
    trainFiles=f.read().splitlines()

with open(offTestSplit) as f:
    testFiles=f.read().splitlines()

with open(offVal1Split) as f:
    valFiles1=f.read().splitlines()

with open(offVal2Split) as f:
    valFiles2=f.read().splitlines()

    
len(trainFiles),len(testFiles),len(valFiles1),len(valFiles2)


(6161, 1861, 900, 940)

In [3]:

df=pd.read_csv(csvpath)
print(" Dataframe shape:",df.shape,df.columns)


 Dataframe shape: (115187, 14) Index(['image_name', 'class', 'width', 'height', 'org_x1', 'org_y1', 'org_x2',
       'org_y2', 'text', 'cropName', 'x', 'y', 'w', 'h'],
      dtype='object')


In [4]:
"""
    below part classify actual training data into IAM handwritten official split
"""

trainCount,testCount,valCount1,valCount2,notFound=0,0,0,0,0
trainCrops,val1Crops,val2Crops,testCrops,notCrops=[],[],[],[],[]
lineNames=[]

for indx,row in df.iterrows():
    
    cropName=row.cropName    
    lineName=cropName[:-3]
    
    #print(" cropName:",cropName," lineName=",lineName)

    lineNames.append(lineName)
    
    if lineName in trainFiles:
        trainCount+=1
        trainCrops.append(lineName)
    elif lineName in valFiles1:
        valCount1+=1
        val1Crops.append(lineName)
        
    elif lineName in valFiles2:
        valCount2+=1
        val2Crops.append(lineName)
    elif lineName in testFiles:
        testCount+=1
        testCrops.append(lineName)
    else:
        notFound+=1
        notCrops.append(lineName)
        #print(" \n indx:",indx," notFound:",notFound," cropName=",cropName,"lineName=",lineName," text:",row.text)
        #input(" check!!!")
        
    if indx%10000==0:
        print(" \n indx:",indx," trainCount:",trainCount," valCount:",valCount1," valCount2:",valCount2," testCount:",testCount," line:",len(set(lineNames))," notFound:",notFound)

print(" 1.total:",trainCount+valCount1+valCount2+testCount+notFound)

trainCrops,val1Crops,val2Crops,testCrops,notCrops=list(set(trainCrops)),list(set(val1Crops)),list(set(val2Crops)),list(set(testCrops)),list(set(notCrops))

print(" 2.total:",trainCount+valCount1+valCount2+testCount+notFound)

 
 indx: 0  trainCount: 0  valCount: 0  valCount2: 0  testCount: 0  line: 1  notFound: 1
 
 indx: 10000  trainCount: 5285  valCount: 645  valCount2: 885  testCount: 931  line: 1171  notFound: 2255
 
 indx: 20000  trainCount: 9314  valCount: 1776  valCount2: 1693  testCount: 2517  line: 2328  notFound: 4701
 
 indx: 30000  trainCount: 13711  valCount: 2413  valCount2: 2305  testCount: 4698  line: 3478  notFound: 6874
 
 indx: 40000  trainCount: 18902  valCount: 3011  valCount2: 2691  testCount: 6269  line: 4615  notFound: 9128
 
 indx: 50000  trainCount: 23859  valCount: 3521  valCount2: 3462  testCount: 7553  line: 5813  notFound: 11606
 
 indx: 60000  trainCount: 27941  valCount: 3996  valCount2: 4454  testCount: 9172  line: 6997  notFound: 14438
 
 indx: 70000  trainCount: 32372  valCount: 4737  valCount2: 5262  testCount: 10766  line: 8142  notFound: 16864
 
 indx: 80000  trainCount: 37073  valCount: 5363  valCount2: 6355  testCount: 12305  line: 9269  notFound: 18905
 
 indx: 90000

In [5]:
trainFiles[0:5]

print(len(trainCrops)+len(val1Crops)+len(val2Crops)+len(testCrops)+len(notCrops))
len(notCrops)
notCrops[:10]

13353


['e01-062-07',
 'l07-172-05',
 'l04-062-02',
 'j06-030-08',
 'h04-071-02',
 'b04-181-07',
 'g06-050j-09',
 'h06-000-00',
 'h06-085-09',
 'g06-045n-06']

In [6]:
"""
    yolov3 localised data crops
    goal is to identify official split at word level
"""
cropBase="/home/aniketag/Documents/phd/PHOSC-Zero-Shot-Word-Recognition-main/data/newFilterData/"

cropsYoloTrainPath=cropBase+"cropsYoloV3Train//"
cropsYoloValPath=cropBase+"cropsYoloV3valid//"
cropsYoloTestPath=cropBase+"cropsYoloV3Test//"

cropsYoloTrain=os.listdir(cropsYoloTrainPath)
cropsYoloVal=os.listdir(cropsYoloValPath)
cropsYoloTest=os.listdir(cropsYoloTestPath)

len(cropsYoloTrain),len(cropsYoloTest),len(cropsYoloVal)


(67595, 7617, 9125)

In [7]:
len(cropsYoloTrain+cropsYoloVal+cropsYoloTest)

84337

In [8]:
"""
    identify overlapping area 
"""

def area(a, b):  # returns None if rectangles don't intersect
    dx = min(a.xmax, b.xmax) - max(a.xmin, b.xmin)
    dy = min(a.ymax, b.ymax) - max(a.ymin, b.ymin)
    if (dx>=0) and (dy>=0):
        return dx*dy

from collections import namedtuple,defaultdict
Rectangle = namedtuple('Rectangle', 'xmin ymin xmax ymax')

d=defaultdict(list)    

"""

    BELOW DICTIONARY d CONTAINS YOLOV3 CROP NAME AND VALUE IS CORRESPONDING
    LINE NUMBER, IDEA IS TO USE THIS LINE NUMBER TO IDENTIFY WHERE YOLOV3 CROPS BELONG 

"""

In [9]:

for indx,imgName in enumerate(cropsYoloTrain+cropsYoloVal+cropsYoloTest):
    
    if ".csv" in imgName:
        continue
    
    orgImgName=imgName.split(".png")[0]+".png"
    temp=df[df["image_name"]==orgImgName]
    
    #print(" indx:",indx," imgName:",imgName," orgImgName:",orgImgName," temp:",temp.shape," cord=",cord," type:",type(cord))

    cord=imgName.split("_[")[1].split("]_")[0]
    cord=eval("["+cord+"]")
    
    x1,y1,x2,y2=cord[0],cord[1],cord[2],cord[3]
    
    ra = Rectangle(x1, y1, x2, y2)
    
    for rowNo,row in temp.iterrows():
        orgX1,orgX2=row.org_x1,row.org_x2
        orgY1,orgY2=row.org_y1,row.org_y2
        
        rb = Rectangle(orgX1, orgY1, orgX2, orgY2)
            
        if area(ra, rb):
            
            d[imgName].append(row.cropName[:-3])# row.text
            
    if indx%5000==0:
        
        print(" indx:",indx," imgName:",imgName," orgImgName:",orgImgName," temp:",temp.shape," cord=",cord," type:",type(cord))

        print(" keys:",len(d.keys()))
    
    
    

 indx: 0  imgName: n06-148.png_45_[753, 2208, 836, 2234]_as_.png  orgImgName: n06-148.png  temp: (93, 14)  cord= [753, 2208, 836, 2234]  type: <class 'list'>
 keys: 1
 indx: 5000  imgName: g03-049.png_6_[671, 1247, 893, 1329]_Noel_.png  orgImgName: g03-049.png  temp: (62, 14)  cord= [671, 1247, 893, 1329]  type: <class 'list'>
 keys: 4929
 indx: 10000  imgName: f01-135.png_81_[313, 2172, 585, 2248]_usually_.png  orgImgName: f01-135.png  temp: (57, 14)  cord= [313, 2172, 585, 2248]  type: <class 'list'>
 keys: 9851
 indx: 15000  imgName: k07-134.png_84_[1059, 1745, 1283, 1831]_when_.png  orgImgName: k07-134.png  temp: (66, 14)  cord= [1059, 1745, 1283, 1831]  type: <class 'list'>
 keys: 14771
 indx: 20000  imgName: j04-012.png_36_[1808, 1628, 2158, 1687]_chemical_.png  orgImgName: j04-012.png  temp: (68, 14)  cord= [1808, 1628, 2158, 1687]  type: <class 'list'>
 keys: 19697
 indx: 25000  imgName: k07-063a.png_69_[905, 1572, 1024, 1641]_that_.png  orgImgName: k07-063a.png  temp: (70, 14)

In [ ]:
trainCrops,val1Crops,val2Crops,testCrops,notCrops

"""

    IDENTIFY WHERE YOLOV3 CROPS BELONG
    
    and create a dictionary which tells where each yoloV3 crop belong.

"""

In [10]:

trainData,val1Data,val2Data,testData,noData=[],[],[],[],[]

for key in d.keys():
    
    offlineName=d[key][0]
    
    if offlineName in trainCrops:
        d[key].append("train")
        trainData.append(key)
    elif offlineName in val1Crops:
        d[key].append("val1")
        val1Data.append(key)
    elif offlineName in val2Crops:
        d[key].append("val2")
        val2Data.append(key)        
    elif offlineName in testCrops:
        d[key].append("test")
        testData.append(key)        
    elif offlineName in notCrops:
        d[key].append("not")
        noData.append(key)    
    
    
print(" ")
    

In [11]:
len(trainData),len(val1Data),len(val2Data),len(testData),len(noData)

(38892, 5851, 6309, 12559, 19421)

In [25]:
import pickle

#a = {'hello': 'world'}

dictDumpPath="/home/aniketag/Documents/phd/PHOSC-Zero-Shot-Word-Recognition-main/data/newFilterData//"

with open(dictDumpPath+'offIamYoloV3CropSplit.pickle', 'wb') as handle:
    pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(dictDumpPath+'offIamYoloV3CropSplit.pickle', 'rb') as handle:
    b = pickle.load(handle)
    
print(d == b)

True


In [ ]:
"""
cropBase="/home/aniketag/Documents/phd/PHOSC-Zero-Shot-Word-Recognition-main/data/newFilterData/"

cropsYoloTrainPath=cropBase+"cropsYoloV3Train//"
cropsYoloValPath=cropBase+"cropsYoloV3valid//"
cropsYoloTestPath=cropBase+"cropsYoloV3Test//"

cropsYoloTrain=os.listdir(cropsYoloTrainPath)
cropsYoloVal=os.listdir(cropsYoloValPath)
cropsYoloTest=os.listdir(cropsYoloTestPath)

len(cropsYoloTrain),len(cropsYoloTest),len(cropsYoloVal)

"""

In [16]:
from shutil import copy2

cropBase="/home/aniketag/Documents/phd/PHOSC-Zero-Shot-Word-Recognition-main/data/newFilterData/"
cropBase2="/home/aniketag/Documents/phd/PHOSC-Zero-Shot-Word-Recognition-main/data/newFilterData2/"

toPath=""
fromPath=""
trainCropCount,valCropCount,testCropCount=0,0,0,


for cropNo,imgCropName in enumerate(val2Data):
        
    if imgCropName in cropsYoloTrain:
            fromPath=os.path.join(cropBase,"cropsYoloV3Train",imgCropName)
        #toPath=os.path.join(cropBase2,"cropsYoloV3Train")
        trainCropCount+=1
        
        
    elif imgCropName in cropsYoloVal:
        fromPath=os.path.join(cropBase,"cropsYoloV3valid",imgCropName)
        #toPath=os.path.join(cropBase2,"cropsYoloV3valid")
        valCropCount+=1
        
    elif imgCropName in cropsYoloTest:
        fromPath=os.path.join(cropBase,"cropsYoloV3Test",imgCropName)
        #toPath=os.path.join(cropBase2,"cropsYoloV3Test")
        testCropCount+=1

    toPath=os.path.join(cropBase2,"cropsYoloV3valid")
    
    copy2(fromPath,toPath)
    
    if cropNo%1000==0:
        print(" cropNo:",cropNo," counts",trainCropCount," ",valCropCount," ",testCropCount,len(os.listdir(toPath)))

    
        
print(" trainCount",trainCropCount," valCount:",valCropCount," testCount:",testCropCount)

 cropNo: 0  counts 1   0   0 5852
 cropNo: 1000  counts 1001   0   0 6852
 cropNo: 2000  counts 2001   0   0 7852
 cropNo: 3000  counts 3001   0   0 8852
 cropNo: 4000  counts 4001   0   0 9852
 cropNo: 5000  counts 5001   0   0 10852
 cropNo: 6000  counts 5045   681   275 11852
 trainCount 5045  valCount: 681  testCount: 583


In [32]:
len(trainData),len(cropsYoloTrain),len(testData)

(38892, 67595, 12559)

In [ ]:
"""
    create .csv file : IAM_trainYoloCropsAug.csv, IAM_testYoloCrops.csv,  IAM_valYoloCrops.csv
"""

In [6]:
import re
import pandas as pd
import os

regex = re.compile('[^a-zA-Z]')
#First parameter is the replacement, second parameter is your input string
#regex.sub('', 'ab3d*E')

cropBase2="/home/aniketag/Documents/phd/PHOSC-Zero-Shot-Word-Recognition-main/data/newFilterData2/"

df=pd.DataFrame(columns=["Image","Word","Writer"])


folderName=["cropYolov3TrainOrigAug","cropYolov3TrainAug","cropsYoloV3valid","cropsYoloV3Test"]
fileName=["IAM_trainYoloCropsOrigAug.csv","IAM_trainYoloCropsAug.csv","IAM_valYoloCrops.csv","IAM_testYoloCrops.csv"]

indx=0
src=os.path.join(cropBase2,folderName[indx])
csvDest=os.path.join(cropBase2,fileName[indx])

allImgCrops=os.listdir(src)

for indx,cropName in enumerate(allImgCrops):
    
    if indx%5000==0:
        print(" indx:",indx," total images:",len(allImgCrops))
        df.to_csv(csvDest,index=False)
    
    word=cropName.split("]_")[1].split("_")[0]
    
    
    word=regex.sub('',word)

    #print(" cropName:",cropName," text=",text)
    
    if len(word):

        df.loc[indx,"Image"]=cropName
        df.loc[indx,"Word"]=word
        df.loc[indx,"Writer"]=""
        """
        if indx==10:
            break
        """
        
df.to_csv(csvDest,index=False)

 indx: 0  total images: 233352
 indx: 5000  total images: 233352
 indx: 10000  total images: 233352
 indx: 15000  total images: 233352
 indx: 20000  total images: 233352
 indx: 25000  total images: 233352
 indx: 30000  total images: 233352
 indx: 35000  total images: 233352
 indx: 40000  total images: 233352
 indx: 45000  total images: 233352
 indx: 50000  total images: 233352
 indx: 55000  total images: 233352
 indx: 60000  total images: 233352
 indx: 65000  total images: 233352
 indx: 70000  total images: 233352
 indx: 75000  total images: 233352
 indx: 80000  total images: 233352
 indx: 85000  total images: 233352
 indx: 90000  total images: 233352
 indx: 95000  total images: 233352
 indx: 100000  total images: 233352
 indx: 105000  total images: 233352
 indx: 110000  total images: 233352
 indx: 115000  total images: 233352
 indx: 120000  total images: 233352
 indx: 125000  total images: 233352
 indx: 130000  total images: 233352
 indx: 135000  total images: 233352
 indx: 140000  to

In [5]:
df.shape,len(allImgCrops),df.columns


((12093, 3), 12093, Index(['Image', 'Word', 'Writer'], dtype='object'))

In [69]:
df.shape,len(allImgCrops),df.columns
imgList=df["Image"].to_list()
delCount=0

for indx, cropName in enumerate(allImgCrops):
    
    if indx%5000==0:
        print(" indx:",indx)
    
    if not cropName in imgList:
        os.remove(src+"//"+cropName)
        delCount+=1
print(" Delete count:",delCount)

 indx: 0
 indx: 5000
 indx: 10000
 Delete count: 14


In [64]:
src

'/home/aniketag/Documents/phd/PHOSC-Zero-Shot-Word-Recognition-main/data/newFilterData2/cropYolov3TrainAug'

In [60]:
df.head(5)

,Image,Word,Writer
0,"a04-043.png_30_[786, 1482, 957, 1523]_main_2_.png",main,
1,"g06-026f.png_6_[1861, 747, 1915, 787]_an_11_.png",an,
2,"e04-103.png_96_[462, 908, 665, 1034]_plank_2_.png",plank,
3,"g06-050a.png_4_[766, 1069, 828, 1133]_he_1_.png",he,
4,"h04-025.png_70_[601, 1794, 652, 1864]_the_3_.png",the,


In [ ]:

x = float("nan")
print(f"It's np.isnan  : {np.isnan(x)}")

In [45]:
import re
import numpy as npprint(f"It's np.isnan  : {np.isnan(x)}")

regex = re.compile('[^a-zA-Z]')


In [57]:
word="1-"
word=str(regex.sub('',word))
print("word:",len(word))

if(word != word):
	print("it's a nan value")
#print(f"It's np.isnan  : {np.isnan(word)}")


word: 0


In [1]:
import os

img=os.listdir("/home/aniketag/Documents/phd/PHOSC-Zero-Shot-Word-Recognition-main/data/newFilterData2/cropYolov3TrainOrigAug")


len(img)


233352

In [9]:
import pandas as pd

df=pd.read_csv("/home/aniketag/Documents/phd/PHOSC-Zero-Shot-Word-Recognition-main/data/newFilterData2/IAM_trainYoloCropsOrigAug.csv")

df = df.sample(frac=1).reset_index(drop=True)

df.to_csv("/home/aniketag/Documents/phd/PHOSC-Zero-Shot-Word-Recognition-main/data/newFilterData2/IAM_trainYoloCropsOrigAug.csv")

df.shape


(231864, 3)

In [8]:
df.head(5)

,Image,Word,Writer
0,"e01-050.png_72_[547, 1835, 668, 1908]_time__1_...",time,NaN
1,"e04-019.png_70_[1760, 1649, 1911, 1739]_then__...",then,NaN
2,"b05-045.png_108_[1192, 1067, 1282, 1144]_he__5...",he,NaN
3,"b01-027.png_87_[1757, 928, 1984, 1009]_duke's_...",dukes,NaN
4,"c04-035.png_42_[942, 1450, 1041, 1535]_the__3_...",the,NaN


In [4]:
import pandas as pd

df=pd.DataFrame(columns=["names"])


In [8]:
lineNumber=0

def load_annotations(annot_path):
    final_annotations = []
    with open(annot_path, 'r') as f:
        txt = f.read().splitlines()
        annotations = [line.strip() for line in txt if len(line.strip().split()[1:]) != 0]

    for lineNumber,annotation in enumerate(annotations):
        # fully parse annotations
        line = annotation.split()
        image_path, index = "", 1
        for i, one_line in enumerate(line):
            if not one_line.replace(",","").isnumeric():
                if image_path != "": image_path += " "
                image_path += one_line
            else:
                index = i
                break
        
        imageName=image_path.split("forms/")[1]
        
        df.loc[lineNumber,"names"]=imageName
        final_annotations.append(imageName)
    return final_annotations

annot_path="/home/aniketag/pred/mnist_test_PhoscIamOfficialSplitForYoloLineLevel1.txt"
load_annotations(annot_path)

df.to_csv("/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3_simula/YOLO/testFiles.csv",index=False)


/usr/bin/sh: 1: pip: not found
